In [1]:
#import sqlparse
#from sqlparse.sql import IdentifierList, Identifier
#from sqlparse.tokens import Keyword, DML
#from query_representation.utils import *

from moz_sql_parser import parse
#from mo_sql_parsing import parse as parse
import os


import pandas as pd
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

In [2]:
QDIR = "./queries/tpcds/all"

In [3]:
FNS = os.listdir(QDIR)

In [4]:
print(len(FNS))
print(FNS[86])

98
14.sql


In [5]:
sqls = []
for fn in FNS:
    if ".sql" not in fn:
        continue
    if "all" in fn:
        continue
    fn = os.path.join(QDIR, fn)
    with open(fn, "r") as f:
        sql = f.read()
#     if "like" in sql.lower():
#         print(sql)
    sqls.append(sql)

In [6]:
print(len(sqls))

98


In [7]:
#print(sqls[2])

In [8]:
# for si, q in enumerate(sqls):
#     if "q4" in q.lower():
#         print(si)
#         print(q)
#         print("********************************")
#         #break

In [9]:
#sql = sqls[50]
sql = sqls[56]

In [10]:
print(sql)









-- q49


select top 100 
 'web' as channel
 ,web.item
 ,web.return_ratio
 ,web.return_rank
 ,web.currency_rank
 from (
 	select 
 	 item
 	,return_ratio
 	,currency_ratio
 	,rank() over (order by return_ratio) as return_rank
 	,rank() over (order by currency_ratio) as currency_rank
 	from
 	(	select ws.ws_item_sk as item
 		,(cast(sum(coalesce(wr.wr_return_quantity,0)) as dec(15,4))/
 		cast(sum(coalesce(ws.ws_quantity,0)) as dec(15,4) )) as return_ratio
 		,(cast(sum(coalesce(wr.wr_return_amt,0)) as dec(15,4))/
 		cast(sum(coalesce(ws.ws_net_paid,0)) as dec(15,4) )) as currency_ratio
 		from 
 		 web_sales ws left outer join web_returns wr  -- skan_memo_stash_49
 			on (ws.ws_order_number = wr.wr_order_number and 
 			ws.ws_item_sk = wr.wr_item_sk)
                 ,date_dim
 		where 
 			wr.wr_return_amt > 10000 
 			and ws.ws_net_profit > 1
                         and ws.ws_net_paid > 0
                         and ws.ws_quantity > 0
                         and ws_sold_dat

In [11]:
p2 = parse(sql)

ParseException: Expecting StringEnd, found "from (\n \ts" (at char 120, (line:18, col:2)	

# Q87 very hard to parse;
### TODO: maybe queries w/ multiple subqueries, we just try to get each select block and parse them individually

In [12]:
sql4 = """-- q87


select count(*) 
from (select distinct c_last_name, c_first_name, d_date
       from store_sales, date_dim, customer  -- skan_memo_stash_87
       where store_sales.ss_sold_date_sk = date_dim.d_date_sk
         and store_sales.ss_customer_sk = customer.c_customer_sk
         and d_month_seq between 1212 and 1212+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from catalog_sales, date_dim, customer
       where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
         and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
         and d_month_seq between 1212 and 1212+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from web_sales, date_dim, customer
       where web_sales.ws_sold_date_sk = date_dim.d_date_sk
         and web_sales.ws_bill_customer_sk = customer.c_customer_sk
         and d_month_seq between 1212 and 1212+11)
cool_cust

"""

sql4 = """-- q87


select count(*) 
from ((select distinct c_last_name, c_first_name, d_date
       from store_sales, date_dim, customer  -- skan_memo_stash_87
       where store_sales.ss_sold_date_sk = date_dim.d_date_sk
         and store_sales.ss_customer_sk = customer.c_customer_sk
         and d_month_seq between 1212 and 1212+11)
       except
      (select distinct c_last_name, c_first_name, d_date
       from catalog_sales, date_dim, customer
       where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
         and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
         and d_month_seq between 1212 and 1212+11))
cool_cust

"""

# sql4 = """-- q87


# select count(*) 
# from (select distinct c_last_name, c_first_name, d_date
#        from store_sales, date_dim, customer  -- skan_memo_stash_87
#        where store_sales.ss_sold_date_sk = date_dim.d_date_sk
#          and store_sales.ss_customer_sk = customer.c_customer_sk
#          and d_month_seq between 1212 and 1212+11)
# cool_cust
# """
parse(sql4)

ParseException: Expecting StringEnd, found "from ((sel" (at char 26, (line:5, col:1)	

# Fix for Q40, change order of from tables

In [13]:

sql3 = """select top 100 
   w_state
  ,i_item_id
  ,sum(case when (cast(d_date as date) < cast ('1998-04-08' as date)) 
        then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_before
  ,sum(case when (cast(d_date as date) >= cast ('1998-04-08' as date)) 
        then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_after
 from
   warehouse  -- skan_memo_stash_40
  ,item
  ,date_dim
  ,catalog_sales left outer join catalog_returns on
       (cs_order_number = cr_order_number 
        and cs_item_sk = cr_item_sk)

 where
     i_current_price between 0.99 and 1.49
 and i_item_sk          = cs_item_sk
 and cs_warehouse_sk    = w_warehouse_sk 
 and cs_sold_date_sk    = d_date_sk
 and d_date >= dateadd(dd, -30, '1998-04-08')
 and d_date <= dateadd(dd, 30, '1998-04-08') 
 group by
    w_state,i_item_id
 order by w_state,i_item_id"""
parse(sql3)

{'top': 100,
 'select': [{'value': 'w_state'},
  {'value': 'i_item_id'},
  {'value': {'sum': {'case': [{'when': {'lt': [{'cast': ['d_date',
           {'date': {}}]},
         {'cast': [{'literal': '1998-04-08'}, {'date': {}}]}]},
       'then': {'sub': ['cs_sales_price',
         {'coalesce': ['cr_refunded_cash', 0]}]}},
      0]}},
   'name': 'sales_before'},
  {'value': {'sum': {'case': [{'when': {'gte': [{'cast': ['d_date',
           {'date': {}}]},
         {'cast': [{'literal': '1998-04-08'}, {'date': {}}]}]},
       'then': {'sub': ['cs_sales_price',
         {'coalesce': ['cr_refunded_cash', 0]}]}},
      0]}},
   'name': 'sales_after'}],
 'from': ['warehouse',
  'item',
  'date_dim',
  'catalog_sales',
  {'left outer join': 'catalog_returns',
   'on': {'and': [{'eq': ['cs_order_number', 'cr_order_number']},
     {'eq': ['cs_item_sk', 'cr_item_sk']}]}}],
 'where': {'and': [{'between': ['i_current_price', 0.99, 1.49]},
   {'eq': ['i_item_sk', 'cs_item_sk']},
   {'eq': ['cs_ware

# replace intersect by union seems to work for now;

In [14]:
sql2 = """select top 100 count(*) from (
    select distinct c_last_name, c_first_name, d_date
    from store_sales, date_dim, customer -- skan_memo_stash_38
          where store_sales.ss_sold_date_sk = date_dim.d_date_sk
      and store_sales.ss_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11) test"""
sql2 = """select top 100 count(*) from (
    select distinct c_last_name, c_first_name, d_date
    from store_sales, date_dim, customer -- skan_memo_stash_38
          where store_sales.ss_sold_date_sk = date_dim.d_date_sk
      and store_sales.ss_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11
  union
    select distinct c_last_name, c_first_name, d_date
    from catalog_sales, date_dim, customer
          where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
      and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11) tmp"""
parse(sql2)

{'top': 100,
 'select': {'value': {'count': '*'}},
 'from': {'value': {'union': [{'select': {'value': {'distinct': [{'value': 'c_last_name'},
        {'value': 'c_first_name'},
        {'value': 'd_date'}]}},
     'from': ['store_sales', 'date_dim', 'customer'],
     'where': {'and': [{'eq': ['store_sales.ss_sold_date_sk',
         'date_dim.d_date_sk']},
       {'eq': ['store_sales.ss_customer_sk', 'customer.c_customer_sk']},
       {'between': ['d_month_seq', 1212, {'add': [1212, 11]}]}]}},
    {'select': {'value': {'distinct': [{'value': 'c_last_name'},
        {'value': 'c_first_name'},
        {'value': 'd_date'}]}},
     'from': ['catalog_sales', 'date_dim', 'customer'],
     'where': {'and': [{'eq': ['catalog_sales.cs_sold_date_sk',
         'date_dim.d_date_sk']},
       {'eq': ['catalog_sales.cs_bill_customer_sk', 'customer.c_customer_sk']},
       {'between': ['d_month_seq', 1212, {'add': [1212, 11]}]}]}}]},
  'name': 'tmp'}}

In [15]:
p2.keys()

NameError: name 'p2' is not defined

In [16]:
#p2["with"]["value"]["from"]["value"]

# Example commands exploring sample query

In [17]:
print(p2.keys())
print(p2["from"])
print("SELECT: ", p2["select"])
print("WHERE: ", p2["where"])

# LT1
#print(p2["where"]["and"][-1]["lt"])
# EQ1, join
print(p2["where"]["and"][0]["eq"])

# element 0: column name / or operations on it, and element 1 is constant / other col;
for x in p2["where"]["and"]:
    print(x)

# name = sales_detail is an alias for a large-ass query / subquery; 
# print(len(p2["with"]["value"]["from"]["value"]["union"]))

# for x in p2["with"]["value"]["from"]["value"]["union"]:
#     print(x.keys())
#     print(x["where"])

NameError: name 'p2' is not defined

In [18]:
def parse_predicates(parsed_sql):
    alldata = {}
    
    def handle_ops(data, key):
        #print("key: ", key)
        #print("data: ", data)
#         if key == "exists":
#             print("TODO: handle exists")
#             return
#         if data[key].keys()[0] == "exists":
#             print("handle not exists")
#             return
        #print(data, key)
        if isinstance(data[key], dict):
            print("FIX handle ops:")
            print(data)
            return
        col1 = data[key][0]
        if isinstance(col1, list) or isinstance(col1, dict):
            # TODO: extract the column thing happening here
            col1 = str(col1)
            tablename = "complex"
        elif "." in col1:
            #print(col1)
            tablename = col1[0:col1.find(".")]
            col1 = col1[col1.find(".")+1:]
            #print(tablename, col1)
        else:
            tablename = "X"
            
        if tablename not in alldata:
            alldata[tablename] = {}
        
        if col1 not in alldata[tablename]:
            alldata[tablename][col1] = {}
            
        if key not in alldata[tablename][col1]:
            alldata[tablename][col1][key] = set()
        
        curkeydata = alldata[tablename][col1][key]
            
        if key == "eq" or key == "neq":
            #assert len(data["eq"]) == 2
            assert len(data[key]) == 2
            #val2 = data["eq"][1]
            val2 = data[key][1]
            if isinstance(val2, int) or isinstance(val2, float):
                curkeydata.add(val2)
            elif isinstance(val2, dict):
                #print(type(val2))
                #print("Val2: ", val2)
                #assert len(val2) == 1
                #print(val2.keys())
                #print(data)
                #alldata[tablename][col1].add(val2["literal"])
                # TODO: can have things like literal etc.
                #print(val2)
                #if val2.keys
                #print(len(val2))
                #print(val2)
                if len(val2) > 1:
                    #print("**SKIPPING handleobj2**")
                    handle_obj(val2)
                else:
                    curkeydata.add(str(val2))
            else:
                #print("JOIN?")
                #print(data)
                #print(val2)
                #assert "." in val2
                return
            
        elif key in ["lt", "gt", "lte", "gte"]:
            assert len(data[key]) == 2
            val2 = data[key][1]
            curkeydata.add(str(val2))
            return
        else:
            print("UNKNOWN")
            print(key)
        
    def handle_obj(obj):
        if isinstance(obj, list):
            for obj1 in obj:
                handle_obj(obj1)

        elif isinstance(obj, dict):
            for k in obj:
                # check for from + where in joins
                #print("k: ", k)
                if k == "where":
                    for k2 in obj[k]:
                        if k2 == "or":
                            pass
                            #print("!!OR!!")
                            #print(obj)
                        elif k2 == "and":
                            for curpred in obj[k][k2]:
                                #print("curpred: ", curpred)
                                for curpredk in curpred:
                                    #print("ck: ", curpredk)
                                    #print("going to call handle_ops1")
                                    
                                    handle_ops(curpred, curpredk)
                        elif k2 in ["eq", "lt", "lte", "gt", "gte"]:
                            handle_ops(obj[k], k2)
                        else:
                            pass
                            #print(k2)
                else:
                    #print("going to call handleobj2")
                    handle_obj(obj[k])
        else:
            pass
                    
    handle_obj(parsed_sql)
    return alldata

In [19]:
for si, sql in enumerate(sqls):
    if "intersect" in sql.lower():
        print(si, " INTERSECT")

32  INTERSECT
84  INTERSECT
89  INTERSECT
91  INTERSECT


In [20]:
ret = parse_predicates(p2)
for k,v in ret.items():
    print(k,v)

NameError: name 'p2' is not defined

In [24]:
from collections import defaultdict
alldfdata = defaultdict(list)
for si, sql in enumerate(sqls):
    #print(sql)
#     if "q80" in sql:
#         print("Skipping Query 89")
#         continue
    
    # intersect queries
    if si in [32, 84, 89, 91]:
        continue
    
    if si in [27, 32, 45, 50, 56, 58, 78, 84, 85, 90, 92]:
    #if si in [27, 45, 50]:   
        continue
        
    if si in [22, 23, 31]:
        continue
    
    print("Query: ", si)
    parsed_sql = parse(sql)
    curdata = parse_predicates(parsed_sql)
#     if si > 1:
#         break
    for inp in curdata:
        for col in curdata[inp]:
            alldfdata["input"].append(inp)
            alldfdata["column"].append(col)
            for op, vals in curdata[inp][col].items():
                if op == "eq":
                    alldfdata["discrete_ops"].append(1)
                else:
                    alldfdata["discrete_ops"].append(0)
                
                if op == "lt":
                    alldfdata["cont_ops"].append(1)
                else:
                    alldfdata["cont_ops"].append(0)
                
                # TODO: parse in better; can we parse these values somehow?
                if op == "in":
                    alldfdata["discrete_ops"].append(1)
                    alldfdata["in_ops"].append(1)
                else:
                    alldfdata["in_ops"].append(0)
                    
                if op == "like":
                    alldfdata["like_ops"].append(1)
                else:
                    alldfdata["like_ops"].append(0)

Query:  0
Query:  1
Query:  2
Query:  3
UNKNOWN
in
UNKNOWN
in
Query:  4
Query:  5
UNKNOWN
in
Query:  6
UNKNOWN
between
Query:  7
UNKNOWN
or
UNKNOWN
or
UNKNOWN
in
UNKNOWN
in
UNKNOWN
between
Query:  8
UNKNOWN
between
UNKNOWN
in
Query:  9
Query:  10
UNKNOWN
in
UNKNOWN
in
UNKNOWN
in
UNKNOWN
in
UNKNOWN
in
UNKNOWN
in
Query:  11
UNKNOWN
between
UNKNOWN
or
UNKNOWN
between
UNKNOWN
or
UNKNOWN
between
UNKNOWN
or
UNKNOWN
between
UNKNOWN
or
UNKNOWN
between
UNKNOWN
or
UNKNOWN
between
UNKNOWN
or
Query:  12
UNKNOWN
in
FIX handle ops:
{'exists': {'select': '*', 'from': {'value': 'catalog_sales', 'name': 'cs2'}, 'where': {'and': [{'eq': ['cs1.cs_order_number', 'cs2.cs_order_number']}, {'neq': ['cs1.cs_warehouse_sk', 'cs2.cs_warehouse_sk']}]}}}
FIX handle ops:
{'not': {'exists': {'select': '*', 'from': {'value': 'catalog_returns', 'name': 'cr1'}, 'where': {'eq': ['cs1.cs_order_number', 'cr1.cr_order_number']}}}}
Query:  13
UNKNOWN
between
Query:  14
UNKNOWN
or
Query:  15
UNKNOWN
in
Query:  16
UNKNOWN
in


ParseException: Expecting select, found ",\n wsr as\n" (at char 2008, (line:73, col:2)	

In [ ]:
df = pd.DataFrame(alldfdata)

In [ ]:
#df["equal_dates"]

In [214]:
HTML(df[["like_ops", "discrete_ops", "cont_ops", "in_ops"]].\
     describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

index,like_ops,discrete_ops,cont_ops,in_ops
count,792.0,792.000000,792.000000,792.0
mean,0.0,0.875000,0.125000,0.0
std,0.0,0.330928,0.330928,0.0
min,0.0,0.000000,0.000000,0.0
50%,0.0,1.000000,0.000000,0.0
90%,0.0,1.000000,1.000000,0.0
99%,0.0,1.000000,1.000000,0.0
max,0.0,1.000000,1.000000,0.0
